In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# %cd '/content/drive/MyDrive/food101/'

In [3]:
!pip install wandb pytorch-lightning -qqq

In [4]:
import os

import pytorch_lightning as pl

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader

from torchmetrics import Accuracy

from torchvision import transforms
from torchvision import Food101
from torchvision.datasets.utils import download_url
import torchvision.models as models
import torch.utils.data as data_utils
from torchmetrics.functional import accuracy
from pytorch_lightning import Trainer

In [11]:
class FoodDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, data_dir: str = './'):
        super().__init__()
        self.data_dir = '/content/drive/MyDrive/'
        self.batch_size = batch_size

        # Augmentation policy for training set
        self.augmentation = transforms.Compose([
              transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
              transforms.RandomRotation(degrees=15),
              transforms.RandomHorizontalFlip(),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        # Preprocessing steps applied to validation and test set.
        self.transform = transforms.Compose([
              transforms.Resize(size=256),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        
        self.num_classes = 101

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        # build dataset
        dataset = Food101(root=self.data_dir, download=True, split="train")

        # split dataset
        # val_size = int(len(dataset)*0.2)
        # train_size = len(dataset)- int(len(dataset)*0.2)
        # print(train_size)
        indices = torch.arange(5000)
        indi = torch.arange(500)

        self.train, self.val = data_utils.Subset(dataset, indices), data_utils.Subset(dataset, indi) #random_split(dataset, [train_size, val_size])

        self.test = Food101(root=self.data_dir, download=True, split="test")
        
        self.test = random_split(self.test, [len(self.test)])[0]

        self.train.dataset.transform = self.augmentation
        self.val.dataset.transform = self.transform
        self.test.dataset.transform = self.transform
        
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, num_workers=2)

In [9]:
# creating the arch and defining the functions for creating the model
class FoodModel_pre_trained(pl.LightningModule):
    def __init__(self, input_shape, num_classes, learning_rate=2e-4, transfer=False):
        super().__init__()
        
        # hyperparams log
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.dim = input_shape
        self.num_classes = num_classes
        
        # transfer learning
        self.feature_extractor = models.resnet18(pretrained=transfer)

        if transfer:
            self.feature_extractor.eval()
            for param in self.feature_extractor.parameters():
                param.requires_grad = False
        
        n_sizes = self._get_conv_output(input_shape)

        self.classifier = nn.Linear(n_sizes, num_classes)

        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = Accuracy()
  
    def _get_conv_output(self, shape):
        batch_size = 1
        tmp_input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self._forward_features(tmp_input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size
        
    def _forward_features(self, x):
        x = self.feature_extractor(x)
        return x
    
    def forward(self, x):
       x = self._forward_features(x)
       x = x.view(x.size(0), -1)
       x = self.classifier(x)
       
       return x
    
    def training_step(self, batch):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)

        acc = self.accuracy(out, gt)

        self.log("train/loss", loss)
        self.log("train/acc", acc)

        return loss
    
    def validation_step(self, batch, batch_idx):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)

        self.log("val/loss", loss)

        acc = self.accuracy(out, gt)
        self.log("val/acc", acc)

        return loss
    
    def test_step(self, batch, batch_idx):
        batch, gt = batch[0], batch[1]
        out = self.forward(batch)
        loss = self.criterion(out, gt)
        
        return {"loss": loss, "outputs": out, "gt": gt}
    
    def test_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean()
        output = torch.cat([x['outputs'] for x in outputs], dim=0)
        
        gts = torch.cat([x['gt'] for x in outputs], dim=0)
        
        self.log("test/loss", loss)
        acc = self.accuracy(output, gts)
        self.log("test/acc", acc)
        
        self.test_gts = gts
        self.test_output = output
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

In [10]:
dm = FoodDataModule(batch_size=32)
model = FoodModel_pre_trained((3, 32, 32), 101, transfer=True)
trainer = pl.Trainer(max_epochs=5, accelerator="gpu")

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(model, dm)
trainer.save_checkpoint("/content/drive/MyDrive/food101/transfer_learning_model.ckpt")

  0%|          | 0/4996278331 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/food101/food-101.tar.gz to /content/drive/MyDrive/food101/
60600


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type             | Params
-------------------------------------------------------
0 | feature_extractor | ResNet           | 11.7 M
1 | classifier        | Linear           | 101 K 
2 | criterion         | CrossEntropyLoss | 0     
3 | accuracy          | Accuracy         | 0     
-------------------------------------------------------
101 K     Trainable params
11.7 M    Non-trainable params
11.8 M    Total params
47.162    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [13]:
trainer.test(model, dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


60600


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.5469703078269958
        test/loss           1.7714015245437622
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/loss': 1.7714015245437622, 'test/acc': 0.5469703078269958}]

# Full Convolution layer

In [ ]:
from torchmetrics.functional import accuracy
class FoodModel_cnn(pl.LightningModule):
    def __init__(self):
        super(FoodModel_cnn, self).__init__()
        self.convolutional1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(12)
        self.convolutional2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.convolutional3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.batch_norm4ch_norm3 = nn.BatchNorm2d(24)
        self.convolutional4 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.batch_norm5 = nn.BatchNorm2d(24)
        n_size = self._get_conv_output((3,32,32))
        print(n_size)
        self.flatten1 = nn.Linear(16224, 512)
        self.flatten2 = nn.Linear(512, 101)

    def _get_conv_output(self, shape):
      batch_size = 1
      input = torch.autograd.Variable(torch.rand(batch_size, *shape))
      output_feat = self._forward_features(input)
      n_size = output_feat.data.view(batch_size, -1).size(1)
      return n_size

    def _forward_features(self, x):
      x = self.pool(F.relu(self.convolutional1(x)))
      x = self.pool(F.relu(self.convolutional2(x)))
      x = self.pool(F.relu(self.convolutional3(x)))
      return x
        
    def forward(self, x):
      x = self._forward_features(x)
      x = x.view(x.size(0), -1)
      x = F.relu(self.flatten1(x))
      x = self.flatten2(x)
      return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr = 1e-7)
        return optimizer
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        accu = accuracy(y_pred, y)
        self.log('accuracy', accu, prog_bar=True)
        return {'loss':loss, 'accuracy':accu}
    def validation_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # self.log('loss', loss, prog_bar=True)
        return {'loss':loss}
    def test_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        self.log("loss",loss)
        accu = accuracy(y_pred, y)
        self.log("accuracy", accu)
        return {'loss':loss, 'accuracy':accu}

In [ ]:
%%time
model = FoodModel_cnn()
module = FoodDataModule(batch_size=32)
# print(model, module)
trainer = Trainer(max_epochs=5)
trainer.fit(model,module)
trainer.save_checkpoint("/content/drive/MyDrive/food101/full_conv_model.ckpt")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


96


INFO:pytorch_lightning.callbacks.model_summary:
   | Name                | Type        | Params
-----------------------------------------------------
0  | convolutional1      | Conv2d      | 912   
1  | batch_norm1         | BatchNorm2d | 24    
2  | convolutional2      | Conv2d      | 3.6 K 
3  | batch_norm2         | BatchNorm2d | 24    
4  | pool                | MaxPool2d   | 0     
5  | convolutional3      | Conv2d      | 7.2 K 
6  | batch_norm4ch_norm3 | BatchNorm2d | 48    
7  | convolutional4      | Conv2d      | 14.4 K
8  | batch_norm5         | BatchNorm2d | 48    
9  | flatten1            | Linear      | 8.3 M 
10 | flatten2            | Linear      | 51.8 K
-----------------------------------------------------
8.4 M     Trainable params
0         Non-trainable params
8.4 M     Total params
33.541    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [15]:
trainer.test(model, module.test_dataloader())

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.01706930622458458
          loss               4.900818824768066
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'loss': 4.900818824768066, 'accuracy': 0.01706930622458458}]

# Basic CNN

In [12]:
class FoodModel_basic(pl.LightningModule):
    def __init__(self):
        super(FoodModel_basic, self).__init__()
        self.l1 = nn.Linear(150528, 101)

    def forward(self, inp):
        return torch.relu(self.l1(inp.view(inp.size(0), -1)))

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr = 1e-7)
        return optimizer
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        accu = accuracy(y_pred, y)
        self.log('accuracy', accu, prog_bar=True)
        return {'loss':loss, 'accuracy':accu}

    def validation_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        # self.log('loss', loss, prog_bar=True)
        return {'loss':loss}

    def test_step(self,batch,batch_idx):
        x,y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred,y)
        self.log("loss",loss)
        accu = accuracy(y_pred, y)
        self.log("accuracy", accu)
        return {'loss':loss, 'accuracy':accu}

In [13]:
%%time
model = FoodModel_basic()
module = FoodDataModule(batch_size=32)
# print(model, module)
trainer = Trainer(max_epochs=5)
trainer.fit(model,module)
trainer.save_checkpoint("lightning_basic.ckpt")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


  0%|          | 0/4996278331 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/food-101.tar.gz to /content/drive/MyDrive/


INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 15.2 M
--------------------------------
15.2 M    Trainable params
0         Non-trainable params
15.2 M    Total params
60.814    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 5min 27s, sys: 1min 38s, total: 7min 6s
Wall time: 27min 49s


In [14]:
trainer.test(model, module.test_dataloader())

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.01706930622458458
          loss               4.900816917419434
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'loss': 4.900816917419434, 'accuracy': 0.01706930622458458}]